## Feature Inject

Implementation of feature inject ([paper](https://arxiv.org/abs/2501.14524)) using hooks.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import json
import os
import torch
import gc
from argparse import Namespace
import random 
import sys
from glob import glob 

from main import main
import yaml
import gc

# Determine device.
device = 'cuda' if torch.cuda.is_available() else 'mps'

def load_config(path="config.yaml"):
    with open(os.path.join("configs", path), "r") as f:
        cfg = yaml.safe_load(f)

    # convert float dtype string → torch dtype
    dtype_map = {
        "float16": torch.float16,
        "float32": torch.float32,
        "bf16": torch.bfloat16
    }
    cfg["float_"] = dtype_map[cfg.get("float_dtype", "float16")]
    return cfg

config = load_config("config_sd14.yaml")
    
model       = config["model"]
variant     = config["variant"]
model_name  = config["model_name"]
image_size  = config["image_size"]
float_      = config["float_"]

scale       = config["scale"]
seed        = config["seed"]
ddim_steps  = config["ddim_steps"]


## Quick use

In [ ]:
from utils.utils_test import generate_triplet
output_folder = f'outputs_{model.split("/")[-1]}'
os.makedirs(output_folder, exist_ok=True)

selected_layers = config['selected_layers']


generate_triplet(
    source_prompt="a dog on a chair",
    target_prompt="a cat in a circus",
    output_folder=f"{output_folder}/triplet_test",
    model=model,
    model_name=model_name,
    variant=variant,
    device=device,
    image_size=image_size,
    selected_skip_keys=selected_layers,
    float_=float_,
    main_fn=main  # your generation function
)


In [ ]:
from utils.utils_test import generate_triplet
output_folder = f'outputs_style_{model.split("/")[-1]}'
os.makedirs(output_folder, exist_ok=True)

selected_layers = config['selected_layers_style']


generate_triplet(
    source_prompt="a cyberpunk dog",
    target_prompt="a cat in a circus Monet style",
    output_folder=f"{output_folder}/triplet_test",
    model=model,
    model_name=model_name,
    variant=variant,
    device=device,
    image_size=image_size,
    selected_skip_keys=selected_layers,
    float_=float_,
    main_fn=main  # your generation function
)


## Evaluation

In [ ]:
args = {}

output_folder = f'outputs_eval_{model.split("/")[-1]'
os.makedirs(output_folder, exist_ok=True)

selected_layers = config['selected_layers']


yml_file = 'data/pnp/wild-ti2i-fake.yaml'

with open(yml_file, "r") as f:
    tests = yaml.safe_load(f)  # This should be a list of dicts (each dict is one test)


os.makedirs(output_folder, exist_ok=True)

for idx, test in enumerate(tests):
    scale = test.get("scale", 7.5)
    seed = test.get("seed", 0)
    ddim_steps = test.get("ddim_steps", 50)
    source_prompt = test.get("source_prompt", "")
    target_prompts = test.get("target_prompts", [])
        
    if 'turbo' in model or 'schnell' in model:
        scale = 0.0
        ddim_steps = 3

    if 'kandinsky' in model:
        ddim_steps = min(ddim_steps, 30)
        
    print(f"\n--- Running test #{idx+1} ---")
    print(f"scale={scale}, seed={seed}, ddim_steps={ddim_steps}")
    print(f"source_prompt='{source_prompt}'")
    print(f"target_prompts={target_prompts}")
    
    for j, target_prompt in enumerate(target_prompts):
        test_tag = f"test{idx+1}_pair{j+1}"

        # ---- Generate A & B once ----
        base_args = {
            'out_dir': output_folder,
            'prompt_A': target_prompt,
            'variant': variant,
            'device': device,
            'prompt_B': source_prompt,
            'image_size': image_size,
            'model': model,
            'model_name': model_name,
            'guidance_scale': 0.0 if ('turbo' in model) or ('schnell' in model) else scale,
            'num_inference_steps': args.get('num_inference_steps', ddim_steps),
            'seed': seed,
            'float': float_,
            'timesteps': [1000, 0],
            'switch_guidance': {},
            'selected_skip_keys': selected_layers[0]
        }

        print(f"Generating A & B for {source_prompt} -> {target_prompt}")
        image_A, image_B, injected_skips, pipe_B = main(Namespace(**base_args), save_results=False, save_b=True)
        image_A.save(os.path.join(output_folder, f"A_{test_tag}.png"))
        image_B.save(os.path.join(output_folder, f"B_{test_tag}.png"))

        # ---- Loop over hyperparameters for C ----
        switch_guidance_list = [{}]#, 0.9, 1.2]
        timestep_list = [[1000, 200]]#, [1000, 100], [1000,200]]

        for skips in selected_layers:
            print(skips)
            skip_tag = f"skips_{'_'.join([s.split('.')[-1] +'_' + s.split('.')[-3] for s in skips])}"

            for sg, ts in zip(switch_guidance_list, timestep_list):
                hyper_args = base_args.copy()
                hyper_args.update({
                    'switch_guidance': sg,
                    'timesteps': ts,
                    'selected_skip_keys': skips
                })

                print(f"Generating C with skip={skip_tag}, SG={sg}, timesteps={ts}")
                image_C = main(Namespace(**hyper_args), injected_skips=injected_skips, pipe_B=pipe_B, save_results=False)
    
                # Save C with detailed name
                sg_tag = f"SG{sg}_T{ts[0]}-{ts[1]}"
                filename = f"C_{test_tag}_{skip_tag}_{sg_tag}.png"
                image_C.save(os.path.join(output_folder, filename))
                
                # Save metadata
                metadata = {
                    "test_pair": test_tag,
                    "source_prompt": source_prompt,
                    "target_prompt": target_prompt,
                    "scale": scale,
                    "seed": seed,
                    "ddim_steps": ddim_steps,
                    "skip_injection": skips,
                    "switch_guidance": sg,
                    "timesteps": ts
                }
                meta_name = f"metadata_{test_tag}_{skip_tag}_{sg_tag}.json"
                with open(os.path.join(output_folder, meta_name), "w") as f:
                    json.dump(metadata, f, indent=4)
        del pipe_B
        gc.collect()

        #time.sleep(5)

    # Save general config for this test
    config = {
        "scale": scale,
        "seed": seed,
        "ddim_steps": ddim_steps,
        "model": model,
        "selected_layers": selected_layers
    }
    config_path = os.path.join(output_folder, "config.json")
    with open(config_path, "w") as f:
        json.dump(config, f, indent=4)
    print(f"Saved experiment config to {config_path}")



--- Running test #1 ---
scale=0.0, seed=50, ddim_steps=3
source_prompt='a photo of a horse in mud'
target_prompts=['a photo of a zebra in the snow', 'a photo of a husky on the grass', 'an oil painting of a white horse', 'a photo of a blue horse toy in playroom']
Generating A & B for a photo of a horse in mud -> a photo of a zebra in the snow

Initializing Pipeline A (skip capture mode)...


Loading pipeline components...: 100%|██████████| 5/5 [00:00<00:00,  8.93it/s]
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


## Test

In [ ]:
sys.path.insert(0, './test')
from prompts import scribblr_prompts, stockimg_prompts, objects, backgrounds, styles
    
test_type = 'controlled'

output_folder = f'outputs_{model.split("/")[-1]}_{test_type}'
os.makedirs(output_folder, exist_ok=True)
num_items = 500 - len(glob(output_folder + '/*'))
print('running', num_items, 'tests')

if test_type == 'wild':
    prompts = scribblr_prompts + stockimg_prompts

    # sample a list of 500 random combinations of prompts (no duplicates)
    sampled_prompts = random.sample([(a, b) for a in prompts for b in prompts if a != b], num_items)         
    print(len(sampled_prompts))
else:
    sampled_prompts = []
    # sample combinations of object, background, style 500 times
    for i in range(num_items): 
        obj = random.choice(objects)
        back = random.choice(backgrounds)
        style = random.choice(styles)
        prompt1 = f"A high-resolution image of a {obj} in the {back}, {style}"
        
        obj_2 = random.choice(objects)
        back_2 = random.choice(backgrounds)
        style_2 = random.choice(styles)
        prompt2 = f"A high-resolution image of a {obj_2} in the {back_2}, {style_2}"
        
        if (obj != obj_2) and (back != back_2) and (style != style_2) and (prompt1, prompt2) not in sampled_prompts:
            sampled_prompts.append((prompt1, prompt2))

    print(len(sampled_prompts))
                
                        
# Generate all commands.
for prompt_A, prompt_B in sampled_prompts:
    sg_tag = f"{'_'.join(prompt_A.split())}_{'_'.join(prompt_B.split())}"
    if not os.path.isdir(os.path.join(output_folder, sg_tag)):
        os.makedirs(os.path.join(output_folder, sg_tag), exist_ok=True)
        print(f"Generating A & B for {prompt_A} -> {prompt_B}")
        base_args = {
            'out_dir': output_folder,
            'prompt_A': prompt_A,
            'variant': variant,
            'device': device,
            'prompt_B': prompt_B,
            'image_size': image_size,
            'model': model,
            'model_name': model_name,
            'guidance_scale': 0.0 if ('turbo' in model) or ('schnell' in model) else scale,
            'num_inference_steps': ddim_steps,
            'seed': seed,
            'float': float_,
            'timesteps': [1000, 0],
            'switch_guidance': {},
            'selected_skip_keys': ''
        }
        image_A, image_B, injected_skips, pipe_B = main(Namespace(**base_args), save_results=False, save_b=True)
        image_A.save(os.path.join(output_folder, sg_tag, f"A.png"))
        image_B.save(os.path.join(output_folder, sg_tag, f"B.png"))
        for layer in injected_skips.keys():
            sample = [layer]
            skip_tag = f"skips_{'_'.join(sample)}"
             
            hyper_args = base_args.copy()
            hyper_args.update({
                    'selected_skip_keys': sample
            })
            print(f"Generating C with skip={skip_tag}")
            image_C = main(Namespace(**hyper_args), injected_skips=injected_skips, pipe_B=pipe_B, save_results=False)
            filename = f"C_{skip_tag}.png"
            image_C.save(os.path.join(output_folder, sg_tag, filename))
        
        for i in range(50):
            n = random.choice([2, 3])
            sample = random.sample(list(injected_skips.keys()), n)
            skip_tag = f"skips_{'_'.join(sample)}"
             
            hyper_args = base_args.copy()
            hyper_args.update({
                    'selected_skip_keys': sample
            })
            print(f"Generating C with skip={skip_tag}")
            image_C = main(Namespace(**hyper_args), injected_skips=injected_skips, pipe_B=pipe_B, save_results=False)
            filename = f"C_{skip_tag}.png"
            image_C.save(os.path.join(output_folder, sg_tag, filename))
        del pipe_B
        gc.collect()
                
                

running 500 tests
479
Generating A & B for A high-resolution image of a neon sign board in the deep space nebula, japanese anime style -> A high-resolution image of a robot assistant in the stormy ocean cliffs, retro futurism style

Initializing Pipeline A (skip capture mode)...


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Saved 52 blocks for injection.


  0%|          | 0/50 [00:00<?, ?it/s]


Running Pipeline C (non-injected mode)...
Saved 52 blocks for injection.


...

  0%|          | 0/50 [00:00<?, ?it/s]

Generating C with skip=skips_unet.down_blocks.2.resnets.0_unet.up_blocks.1.resnets.0

Initializing Pipeline A (skip capture mode)...
Saved 52 blocks for injection.


  0%|          | 0/50 [00:00<?, ?it/s]

Generating C with skip=skips_unet.down_blocks.1.resnets.0_unet.mid_block.attentions.0_unet.up_blocks.0.resnets.1

Initializing Pipeline A (skip capture mode)...
Saved 52 blocks for injection.


  0%|          | 0/50 [00:00<?, ?it/s]